from https://arxiv.org/pdf/2106.11202.pdf, Table 6

In [1]:
import pysm3.units as u

In [2]:
fwhm = [1.57, 1.17, 1.04] * u.arcmin

In [3]:
freqs = [93.8, 147.0, 219.9] * u.GHz

In [4]:
bandwidth = [26.4, 32.5, 53.6] * u.GHz

In [5]:
import numpy as np

In [6]:
from collections import OrderedDict
from astropy import units as u
from astropy.table import QTable

In [7]:
ipac_warning = [
    "Text file in IPAC table format, read with astropy",
    "from astropy.table import QTable",
    "QTable.read('filename.tbl', format='ascii.ipac')",
    f"Instrument model exported from `Arxiv 2106.11202` and from files by Tom Crawford",
]

In [8]:
bands = ["95ghz", "150ghz", "220ghz"]

In [9]:
table = QTable(
    data=[bands, freqs, fwhm, bandwidth],
    names=["band", "center_frequency", "fwhm", "bandwidth"],
)

In [10]:
for key in ["bandpass", "beam"]:
    table[f"{key}_file"] = [f"{key}_{tag}.tbl" for tag in bands]

In [11]:
table["nside"] = 0
table

band,center_frequency,fwhm,bandwidth,bandpass_file,beam_file,nside
,GHz,arcmin,GHz,,,
str6,float64,float64,float64,str19,str15,int64
95ghz,93.8,1.57,26.4,bandpass_95ghz.tbl,beam_95ghz.tbl,0
150ghz,147.0,1.17,32.5,bandpass_150ghz.tbl,beam_150ghz.tbl,0
220ghz,219.9,1.04,53.6,bandpass_220ghz.tbl,beam_220ghz.tbl,0


In [12]:
table.add_index("band")

In [13]:
cd ../../private/202308_panexp_spt3g

/mnt/home/azonca/p/software/s4mapbasedsims/private/202308_panexp_spt3g


In [14]:
table.meta["comments"] = ipac_warning

In [15]:
table.write(
    "spt_instrument_model.tbl",
    format="ascii.ipac",
    overwrite=True,
)

## Bandpasses

In [16]:
!cd ../../private/202308_panexp_spt3g/

In [17]:
bandpass = {}
for freq in bands:
    bandpass[freq] = QTable.read(f"bandpass_{freq.lower()}.txt", format="ascii")
    bandpass[freq].rename_column("freq", "bandpass_frequency")
    bandpass[freq].rename_column("spectrum", "bandpass_weight")
    bandpass[freq].meta["comments"] = ipac_warning
    bandpass[freq].write(f"bandpass_{freq}.tbl", format="ascii.ipac", overwrite=True)

## Beams

In [18]:
beams = QTable.read("compiled_2020_beams.txt", format="ascii", names=["ell"]+bands)

In [19]:
beam = {}
for freq in bands:
    beam[freq] = QTable(data=[beams["ell"], beams[str(freq)]], names=["ell", "B"])
    beam[freq].meta["comments"] = ipac_warning
    beam[freq].write(f"beam_{freq}.tbl", format="ascii.ipac", overwrite=True)